In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize

from consav import linear_interp

import LimitedCommitmentModel as lcm

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)

plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

# c++ settings
do_cpp = True
do_compile = True
do_egm = True
threads = 16

# from EconModel import cpptools
# cpptools.setup_nlopt(folder='cppfuncs/', do_print=True)

In [ ]:
# compile c++ files
if do_cpp & do_compile:
    model = lcm.HouseholdModelClass(par={'do_cpp':do_cpp,'num_Ctot':100})
    model.link_to_cpp(force_compile=do_compile)
    model.cpp.delink()

# Solve and simulate from alternative models

In [ ]:
# settings for models to solve
T = 2
sigma = 0.1
num_love = 21
num_A = 20
num_A_pd = 20

uniform_mat = np.ones((num_A,num_A))/num_A

settings = {'centered_gradient':True, 
       'p_meet': 0.1, 
       'div_A_share': 0.5, 
       'sigma_love':sigma,
       'T':T,'do_cpp':do_cpp,
       'threads':threads, 
       'interp_power':False,
       'num_love': num_love, 
       'do_egm':True, 
       'num_A': num_A_pd, 
       'num_A_pd':num_A_pd,
       'num_Ctot':20,
       'simT':T,
       }


specs = {
    'model 1':{'latexname':'Linear interp', 'par':{**settings, 'interp_method':'linear' }},
    'model 2':{'latexname':'Numerical interp', 'par':{**settings, 'interp_method':'numerical', 'use_guess':False,}},
    # 'model 3':{'latexname':'Deterministic partner assets', 'par':{**settings,'T':2,'interp_method':'numerical'}},
}


In [ ]:
# solve different models
models = {}
for name,spec in specs.items():
    print(f'solving {name}...')
    
    # setup model
    models[name] = lcm.HouseholdModelClass(name=name,par=spec['par'])
    if do_cpp: models[name].link_to_cpp(force_compile=False)
    models[name].spec = spec
    
    # # solve
    %time models[name].solve()

m1 = models['model 1']
m2 = models['model 2']

In [ ]:
guess_mat = m2.sol.C_tot_pd
models['model 3'] = lcm.HouseholdModelClass(name='Numerical interp, correct guess',par={**settings, 'interp_method':'numerical', 'use_guess':True, 'guess_inv': guess_mat})
models['model 3'].link_to_cpp(force_compile=False)
models['model 3'].spec = {'latexname':'Numerical interp, correct guess'}
%time models['model 3'].solve()

In [ ]:
# ## Re-compile and solve
# models['model 1'].cpp.delink()
# models['model 2'].cpp.delink()

# models['model 2'].link_to_cpp(force_compile=True)
# models['model 1'].link_to_cpp(force_compile=False)

# for name in models.keys():

#     print(f'solving {name}...')
#     models[name].allocate()
#     %time models[name].solve()

# m1 = models['model 1']
# m2 = models['model 2']

In [ ]:
# tester numerical inverse marginal utility
par = m1.par
margU = 0.5
iP = 5
guess = 15.0
do_print = True
C = m1.cpp.inv_marg_util_couple(margU,iP,par,guess,do_print)

fig,ax = plt.subplots()
ax.plot(par.grid_marg_u_for_inv[iP],par.grid_inv_marg_u,marker='o');
ax.plot(margU,C,color='red',marker='o')
ax.set(xlim=[0.0,1.0]);
# virker fint her. Men tager meget lang tid at køre... har ikke løst modellen med det
# modellen løser, bare langsomt
# Optimizer kører stort set lige så længe hvis man beder den gætte på det rigtige...

# Solution

In [ ]:
t = 0
iL = 20
iP = 5
var = "C_tot_couple_to_couple"

plt.plot(m1.par.grid_A, getattr(m1.sol,var)[t,iP,iL], label=m1.spec['latexname'], linewidth=6)
plt.plot(m2.par.grid_A, getattr(m2.sol, var)[t, iP, iL], label=m2.spec['latexname'])
plt.legend();

In [ ]:
t = 0
var = "Cw_priv_single_to_single"
grid = "grid_Aw"

plt.plot(getattr(m1.par,grid),getattr(m1.sol,var)[t], linewidth=6, label=m1.spec['latexname'])
plt.plot(getattr(m2.par,grid),getattr(m2.sol, var)[t], linewidth=2,label=m2.spec['latexname'])
plt.legend()


In [ ]:
m = m2
m.simulate()

# identify repartner
ones = np.ones(m.par.simN)
couple = np.column_stack((ones,m.sim.couple[:,:-1]))
single = couple == 0
meet_new = (m.sim.draw_meet < m.par.p_meet)
meet  = np.logical_and(meet_new, single)
o = np.zeros(m.par.simN, dtype=bool)
repartner = np.column_stack((o, np.diff(m.sim.couple) == 1))

Aw = m.sim.A_own[meet]
Ap = m.sim.A_partner[meet]


# make a plot
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1,2,1)
ax.matshow(m.par.prob_partner_A_w)
ax.set_xlabel('grid_Aw')
ax.set_ylabel('grid_Am')
ax.invert_yaxis()


ax = fig.add_subplot(1,2,2)
ax.scatter(Aw, Ap, s=0.1)
ax.set_xlabel('Aw')
ax.set_ylabel('Am')
ax.plot(m.sim.A_own[meet], m.sim.A_own[meet], color='black', linewidth=2);

## Simulation

In [ ]:
var_list = ('power','couple', 'Cw_priv', 'Cm_priv', 'C_tot')
model_list = ('model 1','model 2')
SAVE = False
for init_power_idx in (1, 19):
    for init_love in (0.0, 0.2): 

            for i,name in enumerate(model_list):
                model = models[name]

                # show how starting of in a low bargaining power gradually improves
                model.sim.init_power_idx[:] = init_power_idx
                model.sim.init_love[:] = init_love 
                model.simulate()
                
            for var in var_list:

                fig, ax = plt.subplots()
                
                for i,name in enumerate(model_list):
                    model = models[name]

                    # pick out couples (if not the share of couples is plotted)
                    if var == 'couple':
                        nan = 0.0
                    else:
                        I = model.sim.couple<1
                        nan = np.zeros(I.shape)
                        nan[I] = np.nan

                    # pick relevant variable for couples
                    y = getattr(model.sim,var)        
                    y = np.nanmean(y + nan,axis=0)

                    ax.plot(y,marker=markers[i],linestyle=linestyles[i],linewidth=linewidth,label=model.spec['latexname']);
                    ax.set(xlabel='age',ylabel=f'{var}');

                plt.legend()

                fig_name = f'sim {var} init power{np.round(model.par.grid_power[init_power_idx],2)}_love{init_love}'
                plt.title(fig_name)
                plt.tight_layout()
                if SAVE: plt.savefig(path+fig_name+'.pdf')

In [ ]:
stop

### Differences in simulated behavior...

In [ ]:
model1 = models['model 1']
par1 = model1.par
sol1 = model1.sol
sim1 = model1.sim

model2 = models['model 2']
par2 = model2.par
sol2 = model2.sol
sim2 = model2.sim

init_A = 0.
init_love = 0.02
init_power_idx = 4
init_power = par1.grid_power[init_power_idx]

model1.sim.init_A[:] = init_A
model1.sim.init_love[:] = init_love
model1.sim.init_power_idx[:] = init_power_idx
model1.simulate()

model2.sim.init_A[:] = init_A
model2.sim.init_love[:] = init_love
model2.sim.init_power_idx[:] = init_power_idx
model2.simulate()

Sw1 = sol1.Vw_couple_to_couple-sol1.Vw_couple_to_single[:,np.newaxis, np.newaxis,:]
Sm1 = sol1.Vm_couple_to_couple-sol1.Vm_couple_to_single[:,np.newaxis, np.newaxis,:]

Sw2 = sol2.Vw_couple_to_couple-sol2.Vw_couple_to_single[:,np.newaxis, np.newaxis,:]
Sm2 = sol2.Vm_couple_to_couple-sol2.Vm_couple_to_single[:,np.newaxis, np.newaxis,:]

In [ ]:
# interpolate spousal surplus for sim1
simSw1 = np.zeros((par1.simN,par1.simT))
simSm1 = np.zeros((par1.simN,par1.simT))
for i in range(par1.simN):
    for t in range(par1.simT):
        A = sim1.A[i,t]
        love = sim1.love[i,t]
        power = sim1.power[i,t]
        simSw1[i,t] = linear_interp.interp_3d(par1.grid_power,par1.grid_love,par1.grid_A,Sw1[t], power, love, A)
        simSm1[i,t] = linear_interp.interp_3d(par1.grid_power,par1.grid_love,par1.grid_A,Sm1[t], power, love, A)

# interpolate spousal surplus for sim2
simSw2 = np.zeros((par2.simN,par2.simT))
simSm2 = np.zeros((par2.simN,par2.simT))
for i in range(par2.simN):
    for t in range(par2.simT):
        A = sim2.A[i,t]
        love = sim2.love[i,t]
        power = sim2.power[i,t]
        simSw2[i,t] = linear_interp.interp_3d(par2.grid_power,par2.grid_love,par2.grid_A,Sw2[t], power, love, A)
        simSm2[i,t] = linear_interp.interp_3d(par2.grid_power,par2.grid_love,par2.grid_A,Sm2[t], power, love, A)

In [ ]:
A = sim1.A[0,0]
power = sim1.power[0,0]

sol_power = np.zeros(par1.num_love)
for iP in range(par1.num_love):
    love = par1.grid_love[iP]
    sol_power[iP] = linear_interp.interp_3d(par1.grid_power,par1.grid_love,par1.grid_A,sol1.power[1], power, love, A)

In [ ]:
I1 = np.argsort(sim1.love[:,1])
I2 = np.argsort(sim2.love[:,1])

plt.plot(sim1.love[I1,1], sim1.power[I1,1], label='update power')
plt.plot(sim2.love[I2,1], sim2.power[I2,1], label='interp power')
plt.scatter(par1.grid_love, sol_power, color='green', label='solution')

# label axes
plt.xlabel('love')
plt.ylabel('power')
plt.legend();

In [ ]:
I1 = np.argsort(sim1.love[:,1])
I2 = np.argsort(sim2.love[:,1])


plt.plot(sim1.love[I1,1], sim1.power[I1,1], label='update power')
plt.plot(sim2.love[I2,1], sim2.power[I2,1], label='interp power')
plt.scatter(par1.grid_love, sol_power, color='green', label='solution')

# label axes
plt.xlabel('love')
plt.ylabel('power')
plt.legend()

# limit x axis
plt.xlim([-0.2,-0.05])

### Time

In [ ]:
%timeit models['model 1'].simulate()
%timeit models['model 2'].simulate()